In [ ]:
import os
os.getcwd()
os.chdir("..")

In [ ]:
os.getcwd()


In [ ]:
# Optimizing the chucking process
import os
import pdfplumber
import re




text_list = []

with pdfplumber.open("PDF_docs/doc_0.pdf") as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            text_list.append(text)
            text = text.join(text_list)
text





In [ ]:
parts = re.split(r"\breferences\b", text, flags=re.IGNORECASE, maxsplit=1)

body = parts[0].strip()  # Everything before "References"
reference = parts[1].strip() if len(parts) > 1 else ""  # Everything after "References"



In [ ]:
reference

In [ ]:
# Get meta data

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_2.pdf")
pdf.metadata

In [ ]:
pdf = pdfplumber.open("PDF_docs/doc_0.pdf")
metadata = pdf.metadata  # Extrahiere Metadaten
if metadata and "Title" in metadata and metadata["Title"]:
    print( f"Titel: {metadata['Title']}")
else: 
    print(False)

In [ ]:
with pdfplumber.open("PDF_docs/doc_1.pdf") as pdf:
    metadata = pdf.metadata
    print(metadata)

# Create Chroma datebank
---

In [1]:
from ollama import ChatResponse, chat
from langchain_ollama import OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_chroma import Chroma
import chromadb

# python PDFProcessing/PDFProccesing.py > outputs/refoutput.txt

In [2]:
import os
os.chdir("..")

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import OllamaEmbeddings
import chromadb
from langchain_chroma import Chroma

# # 📌 Schritt 1: PDFs laden
# pdf_dir = "PDF_docs/"
# pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]  # Nur PDFs

# all_docs = []
# for pdf_file in pdf_files:
#     pdf_path = os.path.join(pdf_dir, pdf_file)
#     loader = PyPDFLoader(pdf_path)
#     docs = loader.load()
#     all_docs.extend(docs)  # Dokumente speichern

# print(f"✅ {len(all_docs)} Dokumente geladen.")

# # 📌 Schritt 2: Embeddings erstellen
# embeddings = OllamaEmbeddings(model="llama3.2:latest")  # Korrekte Embeddings

# # 📌 Schritt 3: ChromaDB persistent speichern
# persistent_client = chromadb.PersistentClient(path="./chroma_langchain_db")  # Verzeichnis für Speicherung
# collection = persistent_client.get_or_create_collection("collection_name")

# 📌 Schritt 4: Dokumente zu ChromaDB hinzufügen
# doc_texts = [doc.page_content for doc in all_docs]  # Extrahiere den Text
# doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs

# collection.add(ids=doc_ids, documents=doc_texts)  # Speichern in ChromaDB

# # 📌 Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
# vector_store_from_client = Chroma(
#     persist_directory="./chroma_langchain_db",
#     collection_name="collection_name",
#     embedding_function=embeddings,
# )

# print("✅ ChromaDB erfolgreich mit PDFs initialisiert!")


In [3]:
file_path = "PDF_docs/"

def save_doc(file_path):
    # Wir laden die pdf dokumente aus dem direcotire nur die mit .pdf enden
    pdf_files = [f for f in os.listdir(file_path) if f.endswith(".pdf")]  # Nur PDFs

    all_docs = [] # initalisieren eine leere liste
    for pdf_file in pdf_files:
        pdf_path = os.path.join(file_path, pdf_file)
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()
        all_docs.extend(docs)  # Dokumente speichern
        # Speichern all dokumente in die liste all_docs
    return all_docs
all_docs = save_doc(file_path=file_path)
len(all_docs)

58

In [4]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(all_docs)
texts

[Document(metadata={'source': 'PDF_docs/doc_0.pdf', 'page': 0, 'page_label': '2207'}, page_content='T h e n e w  e ng l a n d  j o u r na l  o f m e dic i n e\nn engl j med 366;23 nejm.org june 7, 2012 2207\n                   anniversary article\nTwo Hundred Years of Cancer Research\nVincent T. DeVita, Jr., M.D., and Steven A. Rosenberg, M.D., Ph.D.\nFrom the Yale Comprehensive Cancer \nCenter and Smilow Cancer Hospital at \nYale–New Haven, Yale University School \nof Medicine, and Yale University School \nof Public Health — all in New Haven, CT \n(V.T.D.); the National Cancer Institute, \nNational Institutes of Health, and the \nUniformed Services University of the \nHealth Sciences School of Medicine — \nall in Bethesda, MD (S.A.R.); and George \nWashington University School of Medi-\ncine, Washington, DC (S.A.R.). Address \nreprint requests to Dr. DeVita at the  \nYale Comprehensive Cancer Center and \nSmilow Cancer Hospital at Yale–New  \nHaven, 333 Cedar St., PO Box 208028, \nNew

In [5]:
def embedding():
    embeddings = OllamaEmbeddings(model="granite-embedding:30m")  # Korrekte Embeddings
    return embeddings

embeddings = embedding() 
embeddings

OllamaEmbeddings(model='granite-embedding:30m', base_url=None, client_kwargs={})

In [20]:
def persistent_clientChroma():
    persistent_client = chromadb.PersistentClient(path="./chroma_db")  # Verzeichnis für Speicherung
    heartbeat = persistent_client.heartbeat()
    print(f"ChromaDB läuft. Serverzeit: {heartbeat}")

    collection = persistent_client.get_or_create_collection("collection_name")
    return collection, heartbeat

collection = persistent_clientChroma()
collection

ChromaDB läuft. Serverzeit: 1740503222709945201


(Collection(name=collection_name), 1740503222709945201)

In [ ]:
import uuid
ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in all_docs]
unique_ids = list(set(ids))
print(len(unique_ids))
unique_ids


In [ ]:
def add_doc_to_Chroma_():
    doc_texts = [doc.page_content for doc in all_docs]  # Extrahiere den Text
    #doc_ids = [f"doc_{i}" for i in range(len(doc_texts))]  # Einzigartige IDs
    collection = persistent_clientChroma()
    collection_db = collection.add(ids=unique_ids, documents=doc_texts)  # Speichern in ChromaDB
    return collection_db
collection_db = add_doc_to_Chroma_()
collection_db

In [ ]:
def vector_store_from_clientChroma():
    # Schritt 5: Chroma-VectorStore mit gespeicherten Daten initialisieren
    vector_store_from_client = Chroma(
        persist_directory="./chroma_langchain_db",
        collection_name="collection_name",
        embedding_function=embeddings,
    )
    return vector_store_from_client

In [ ]:
vector_store_from_client= vector_store_from_clientChroma()
vector_store_from_client

In [ ]:
def get_all_metadatas():
    results = collection.get()
    
    for i in range(len(results["ids"])):
        print(f"📄 Dokument-ID: {results['ids'][i]}")
        print(f"📜 Inhalt: {results['documents'][i][:100]}...")  # Nur ersten 100 Zeichen anzeigen
        print(f"🏷️ Metadaten: {results['metadatas'][i]}")
        print("-" * 50)
print(get_all_metadatas())

In [ ]:
def query_documents_with_metadata(query_text, top_k=3):
    
    results = collection.query(
        query_texts=[query_text],
        n_results=top_k
    )

    for i in range(len(results["documents"][0])):
        print(f"📌 Ähnlichkeits-Treffer {i+1}")
        print(f"📜 Inhalt: {results['documents'][0][i][:100]}...")
        print(f"🏷️ Metadaten: {results['metadatas'][0][i]}")
        print("-" * 50)
print(query_documents_with_metadata(query_text="Hallo"))

In [ ]:
# results = vector_store_from_client.as_retriever(search_type='similarity',
#     k=2
# )
# print(results)
# question = ' to a Gaussian process'
# results.invoke(question)

In [ ]:
print(len(embeddings.embed_query("test")))  # Gibt die Anzahl der Dimensionswerte aus


In [ ]:
print(all_docs[1].metadata)  # Prüfen, welche Metadaten die Dokumente haben


In [ ]:
print(collection.count())  # Sollte die Anzahl der gespeicherten Dokumente anzeigen


In [ ]:
print(embeddings.embed_query("Germany"))


In [ ]:
results = collection.query(
    query_texts=["CHRISTOPH REICHARD"],
    n_results=2
)

results

In [ ]:
collection.query(
    query_embeddings=embeddings.embed_query("CHRISTOPH REICHARD"),
    n_results=10,
    where={"metadata_field": ""},
    where_document={"$contains":"Germany"}
)